# About the code

This is a main jupyter notebook used to initiate / control the OPO setup and to acquire data. The overall experimental setup is explained in Fig. S7 of supplementary information.

# Import python packages & set global variables

In [8]:
## Import all the necessary libraries (to install use pip or use directly the github of said libraries or use requirements.txt)

import pyvisa #package to control instrument
import numpy as np
import matplotlib.pyplot as plt
import time,sys
from math import sqrt
from ctypes import *
import scipy.signal
from copy import copy, deepcopy
from datetime import datetime
import win32com.client
import toptica.lasersdk.client
from toptica.lasersdk.client import Client, NetworkConnection
from toptica.lasersdk.client import UserLevel, Subscription, Timestamp, SubscriptionValue
from collections import Counter
import os
from datetime import date
from nistrng import *
import scipy.io
import thorlabs_apt
import pylablib.devices.Thorlabs
%matplotlib notebook     
#Enable the plotting on Jupyter Notebook

Specific values asssigned here can be adjusted locally in later cells

In [4]:
#Input variables:

Data_Path = #"path_to_save_data"

#VISA instrument names
fgen_visa = #visa name of funciton generator EX. "GPIBO::12:INSTR"
osc_visa = #visa name of oscilloscope
PID_visa = #visa name of PID
MotionControll_visa= #visa name of Rotation Angle Filter 


# Function Generator sweeping variables
# These variables set the values of the function generator in order to sweep the cavity for resonance peaks

func_symmetry = 50 # symmetry of functions (e.g 100, 50)
phase = 0 # Phase angle
volt_high = 5 # Maximum voltage of function (in volts)
volt_low = 0 # Minimum voltage of function (in volts)
freq = 100 # Frequency of function (in Hertz)
func_channel= 2  # Channel used by the sweep in the function generator
#waveform is hardcoded in the programm

# Value for the modulation

modulation_simmetry = 50
modulation_phase = 0 #Phase angle
modulation_volt_high = 1.60 # Maximum voltage of function (in volts)
modulation_volt_low = -5.650 #Minimum voltage of function (in volts)
modulation_freq = 10e3 # Frequency of function (in Hertz)
modulation_channel= 1 #Channel used by the Modulation in the function generator
#waveform is hardcoded in the programm

# Oscilloscope variables
# These variables set the oscilloscope display and data recording settings

opo_ch_source = 1 # stores the channel input for the opo signal (e.g 1)
error_ch_source = 4 # stores the channel input for the error signal (e.g 1)
volt_ch_source = 3 # stores the channel input for the driving voltage signal (e.g 1)
modulation_ch_source = 2 # stores the channel input for the driving modulation signal (e.g 1)
opo_volts_per_div = 0.05 # sets the vertical voltage sscale of oscilloscope opo input(in volts)
error_volts_per_div = 0.02 # sets the vertical voltage sscale of oscilloscope error signal input (in volts)
voltage_volts_per_div = 0.2 # sets the vertical voltage sscale of oscilloscope voltage signal input(in volts)
trigger_level = 0.6 # sets the trigger level on oscilloscope in order to trigger the scope and collect data (in volts)
nbr_data_points= 200000 # Number of data points in the oscilloscope

#PID controller parameters
setpoint_mode = 0 # sets the set point mode, 0 for internal, 1 for external
offset = 0.3 #sets the offset of the PID box (in volts)
max_out=offset*2 # Maximum voltage given by the PID (in volts)
min_out=0  # Minimum voltage given by the PID (in volts)
p_gain = 0.4 #sets the proportional gain of PID 
i_gain = 14.8E3 # sets the integral gain of PID
#Initial PID BOX variables

#These variables set the initial mode and values of the PID box
manual_output = offset #The voltage output in volts of the PID box in manual mode

# Component controller

This section define all the necessary tools to control the electric devices connected to the computer.

### PID Controller

In [10]:
# Define a class for a PID object that can control the PID

class PID :
   
    def __init__(self, path=PID_visa): 
        
        # path : String like pointing to the visa address of the PID
        # Initiate an instance of the PID, use the visa adress "path"
        
        rm = pyvisa.ResourceManager()
        self.conn = rm.open_resource(path)
        self.dType = self.conn.query('*IDN?')
        print ("PID connected : " + self.dType)  #Indicate to the user that the PID is connected
            
    def set_offset(self,V=0.64):
        
        # V : Float for the voltage 
        # Set the offset of the PID to "V" (in V)
        
        self.conn.write("SNDT 1, 'AMAN 0'")
        self.conn.write("SNDT 1, 'MOUT "+str(V)+"'") 
    
    def stop(self):
        #Stop the PID, no argument
        
        self.conn.write("SNDT 1, 'AMAN 0'")
        
    def setup_control(self,setpoint_mode=0, offset=3, p_gain=0.2, i_gain=92):
        # setpoint_mode : int defining the setpoint mode (0= internal, 1=external)
        # offset : Float defining the offset voltage (in V)
        # p_gain : Float defining the p_gain of the PID
        # i_gain : Float defining the i_gain of the PID
        # Setup all the PID values above before any measurements 
        
        self.conn.write("SNDT 1, 'INPT "+ str(setpoint_mode) +"'")
        self.conn.write("SNDT 1, 'OFST "+str(offset) + "'")
        self.conn.write("SNDT 1, 'OCTL 1'")
        self.conn.write("SNDT 1, 'GAIN "+str(p_gain)+"'")
        self.conn.write("SNDT 1, 'PCTl 1'")
        self.conn.write("SNDT 1, 'ICTL 1'")
        self.conn.write("SNDT 1, 'INTG "+str(i_gain)+"'")
        
    
    def control(self,int_set):
        # int_set : Float defining the internal setpoint (in V)
        # Enable the control by the PID to the setpoint "int_set" (in V)
        
        int_set=int(1000*int_set)/1000
        self.conn.write("SNDT 1, 'SETP "+ str(int_set) +"'")
        self.conn.write("SNDT 1, 'AMAN 1'")
    
    def get_out (self):
        # Get the voltage send by the PID (in V) 
        
        self.conn.write("SNDT 1, 'OMON?'")
        try:
            a=float(self.conn.query("GETN? 1, 120")[6:-3])
        except:
            a=self.get_out()
        finally:
            return(a)     

### FunctionGenerator

In [ ]:
# Define a class for a FunctionGenerator object that can control the FunctionGenerator

class FunctionGenerator :
    
    def __init__(self, path=fgen_visa):
        # path : String like pointing to the visa adress of the FunctionGenerator
        # Initiate an instance of the FunctionGenerator, use the visa adress "path"
        
        rm = pyvisa.ResourceManager()
        self.conn = rm.open_resource(path)
        self.dType = self.conn.query('*IDN?')
        print ("Function generator Connecetd : " +self.dType) #Indicate to the user that the Function Generator is connected
            
    def set_function_ramp(self,func_channel=2,sym=100,phase=-179,upV=3,lowV=0,freq=100): #create triangle wave
        self.conn.write("OUTP"+str(func_channel)+":LOAD INF")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion RAMP")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion:RAMP:SYMMetry "+str(sym))
        self.conn.write("UNIT:ANGL DEG")
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:HIGH "+str(upV))
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:LOW "+str(lowV))
        self.conn.write("SOURce"+str(func_channel)+":FREQuency "+str(freq))
    
    def set_function_DC(self,V,func_channel=2): #create dc wave
        self.conn.write("OUTP"+str(func_channel)+":LOAD INF")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion DC")
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:OFFSet "+str(V))
    
    def set_function_square(self,func_channel=1,sym=50,phase=0,upV=0,lowV=-2.4,freq=3e3): #create square wave
        self.conn.write("OUTP"+str(func_channel)+":LOAD INF")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion SQUare")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion:RAMP:SYMMetry "+str(sym))
        self.conn.write("UNIT:ANGL DEG")
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:HIGH "+str(upV))
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:LOW "+str(lowV))
        self.conn.write("SOURce"+str(func_channel)+":FREQuency "+str(freq))
    
    def set_function_sin(self,func_channel=1,phase=0,upV=0,lowV=-2.4,freq=3e3): #create sin wave
        self.conn.write("OUTP"+str(func_channel)+":LOAD INF")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion SIN")
        self.conn.write("UNIT:ANGL DEG")
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:HIGH "+str(upV))
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:LOW "+str(lowV))
        self.conn.write("SOURce"+str(func_channel)+":FREQuency "+str(freq))
    
    def stop(self,func_channel=2):
        # func_channel : int indicating the channel which we want to stop
        # Stop the channel "func_channel"
        
        self.conn.write("OUTPut"+str(func_channel)+" OFF")
    
    def start(self,func_channel=2):
        # func_channel : int indicating the channel which we want to stop
        # Start the channel "func_channel"
        
        self.conn.write("OUTPut"+str(func_channel)+" ON")

### Oscilloscope

In [ ]:
# Define a class for a oscilloscope object that can control the oscilloscope

class Oscilloscope:
    
    def __init__(self, path=osc_visa):
        # path : String like pointing to the visa adress of the Oscilloscope
        # Initiate an instance of the Oscilloscope, use the visa adress "path"
        
        scope=win32com.client.Dispatch("LeCroy.ActiveDSOCtrl.1") 
        scope.MakeConnection(path)
        if scope.WriteString('*IDN?',True):
            print("Oscilloscope connected : " +scope.ReadString(500)) #Indicate to the user that the Function Generator is connected
        self.conn=scope
    
    def acquire(self,channel):
        # channel : Int indicating the channel of the oscilloscope we want to acquire
        # Acquire the channel "channel" of the oscilloscope
        # Return two arrays, the first contating the time value (in s) and the second containing the voltage value of the channel (in V)
        
        chan='C'+str(channel)
        wav=self.conn.GetScaledWaveformWithTimes(chan,999999,True)
        wav=np.array(wav)
        return(wav)
        tim=[]
        amp=[]
        for k in range (len(wav[1])):
            tim.append(wav[0][k])
            amp.append(wav[1][k])
        return(tim,amp) 
    
    def set_time(self,dT,data_points=nbr_data_points):
        # dT : Float, Time period of one division of the oscilloscope (in S)
        # data_points : Int, Number of total Data Points of the oscilloscope
        # Set the the time period of one division to dT and the total number of points to data_points
        # Return the time period between two consecutive points of the oscilloscope
        
        self.conn.WriteString("SCLK ECL,"+str(nbr_data_points),True)
        self.conn.WriteString('TIME_DIV ' + str(dT),True)
        return (dT*10/data_points)
    
    def set_memory(self,N):
          
        # Memory: Int, Number of memory can be used for data acquisition
        # N: Increase the memory by ratio N. 
        # Possible memory size: 500, 1000, 2500, 5000, 10k, 25k, 50k, 100k, 250k, 500k, 1M, 2.5M, 5M, 10M, 25M
        # Changing the timescale would not automatically increase the memory size, we might have chances to lose resolution of bitstream if we increase the bit by simply increasing the time scale   
        
        self.conn.WriteString("Memory_SIZe?", True)
        current_memory_string=o.conn.ReadString(500) # get current memory size as a string
        self.conn.WriteString("Time_DIV?", True) 
        current_time_div_string=o.conn.ReadString(500).split()[1] # get current time division as a string
        current_time_div_num=float(current_time_div_string)
        current_memory_int=0
        txt=current_memory_string.split()[1] # current_memory_string is given as 10k, 1M. Therefore, we have to separate 10 and k, 1 and M to get numerical value
        emp_str=""
        for m in txt:
            if m.isdigit():
                emp_str+=m # get only numbers, not letters
        if txt != "2.5M": # we separate the case of 2.5M, since this is the only case that includes "point" in the string
            if txt.find("K")!=-1: # returns -1 when there is no specific letter
                current_memory_int=int(float(emp_str))*1000
            elif txt.find("M")!=-1:
                current_memory_int=(float(emp_str))*1000000
            else:
                current_memory_int=int(float(emp_str))
        else:
            current_memory_int=2500000
        
        sampling_rate=current_memory_int/current_time_div_num
        update_time=current_time_div_num*N
        update_memory=sampling_rate*update_time
        temp="MSIZ "
        self.conn.WriteString(temp+str(update_memory),True) #e.g. MSIZ 1000
        self.conn.WriteString('TIME_DIV ' + str(update_time),True)

### Laser

In [8]:
# Connect the Toptica laser 

with toptica.lasersdk.client.Client(toptica.lasersdk.client.NetworkConnection('your_laser_network')) as client:
        print("=== Connected Device ===")
        print("This is a {} with serial number {}.\n".format(client.get('system-type'), client.get('serial-number')))
        #client.exec('system-service-report:service-report', output_type=bytes)

=== Connected Device ===
This is a FFultra780 with serial number FFultra780_01059.



### Motion Controller

In [ ]:
# Define a class for a MotionController object that can control the Rotation Stage of the green filter

class MotionController:
    
    def __init__(self, path=MotionControll_visa):
        # path : String like pointing to the visa adress of the MotionController
        # Initiate an instance of the MotionController, use the visa address "path"
        
        rm = pyvisa.ResourceManager()
        self.conn = rm.open_resource(path)
        try :
            self.conn.query("1MD?")
            print("Motion Controller Connected")  # Indicate to the user that the Function Generator is connected
        except:
            pass
     
        
    def moveto(self,angle):
        # angle : Float, angle to be moved at (degree)
        # Move the MotionController to the absolute angle "angle", stop the workflow while not finished
        
        self.conn.write("1PA+"+str(angle))
        while self.conn.query("1MD?")[0]=='0':
            pass
    
    def moveby(self,angle):
        # angle : Float, angle to be moved by (degree)
        # Move the MotionController by the angle "angle", stop the workflow while not finished
        
        self.conn.write("1PR+"+str(angle))
        while self.conn.query("1MD?")[0]=='0':
            pass

### Rotation Stage

In [ ]:
# Define a class for a RotationStage object that can control the Rotation Stage of the Waveplate in front of the bias

class RotationStage:
    
    def __init__(self):
        # path : String like pointing to the visa adress of the RotationStage
        # Initiate an instance of the RotationStage, use the visa adress "path"
        
        try:
            motor = thorlabs_apt.Motor(thorlabs_apt.list_available_devices()[0][1])
            motor.enable()
            motor.move_home(blocking=True)
            motor.set_stage_axis_info(0,100.0,2,0.5)
            print("Motion Controller Connected")
            self.conn=motor
        except:
            pass
     
        
    def moveto(self,angle):
        # angle : Float, angle to be moved at (degree)
        # Move the MotionController to the absolute angle "angle", stop the workflow while not finished
        
        pos=(angle/36)%10.0
        self.conn.move_to(pos,blocking=True)
            
    def moveby(self,angle):
        # angle : Float, angle to be moved by (degree)
        # Move the MotionController by the angle "angle", stop the workflow while not finished
        
        pos=(angle/36)%10.0
        self.conn.move_by(pos,blocking=True)

# Connectivity check

### Following functions are used to check whether electronic devices are connected properly or not

In [ ]:
# Try to create an instance of all the previously defined electric devices, no input needed.
# Use the variables m,o,p,f,r,pm for : the MotionController, the Oscilloscope, the PID, the FunctionGenerator, RotationStage, and the Piezo Motor Stage
# Instances returned to be used later


def ConnectAll():
    m,o,p,f,r,pm=None,None,None,None,None,None #Initiate the value if no connection is found
    try:
        m=MotionController()  # Connect the Motion Controller 
    finally:
        try:
            o=Oscilloscope() # Connect the Oscilloscpe
        finally:
            try:
                p=PID() # Connect the PID 
            finally:
                try:
                    f=FunctionGenerator() # Connect the Function Generator
                finally:
                    try:
                        r=RotationStage() # Connect the Rotation Stage
                    finally:
                        try:
                            pm=pylablib.devices.Thorlabs.kinesis.KinesisPiezoMotor("your_part_number",default_channel=1) # Connect the Piezo Motor Stage
                        finally:
                            return (m,o,p,f,r,pm)

In [ ]:
# Print which visa objects are connected to the computer
# Useful to check which device is connected in case of missfunction

def WhatConnected():
    rm=pyvisa.ResourceManager() #Access to connected visa
    print(rm.list_resources())

# Finding OPO modes

To find the proper OPO mode, we apply triangular voltage to the piezo attached to one of the mirrors forming the cavity. The following functions are used to determine the peak of the initial sweep and then get their variances. Therefore, one can guess which voltage should be used to lock on a mode.

### Basic functions

In [ ]:
# L: Float List, List of time for which one want the triangle signal to be constructed (in s) 
# V: Float, Max value of the signal - Min Value of the signal (in V)
# off: Float, Offset of the signal (in V)
# phase: Float, shift at the origin of the signal (in s)
# freq : Float, Frequency of the signal
# Create a triangle signal for the time in L, with every parameters above
# Return a Float List containing the signal

def triangle(L,V,off,phase,freq):
    y=[]
    for x in L:
        x=abs(x+phase)%freq
        if x>freq/2:
            y.append(off+V-2*V*(x-freq/2)/freq)
        else:
            y.append(off+2*V*x/freq)
    return(y)

In [ ]:
# x : Float, x value of the gaussian 
# sigma : Float, std of the gaussian
# Return a Float value of a gaussian function of std "sigma" evaluated in "x"

def gauss(x, sigma):
    return np.exp(- x** 2 / (2 * sigma ** 2))

### Processing triangular sweep

In [ ]:
#lis : Float List, Triangle Signal to process
# Use a gaussian filter to convolve the signal and smooth it to erase noise then get all the peaks of the triangle
# Goal is to get the different sweeps to seperate the signal
# Return the peaks list

def ProcessTriangle(lis):
    sigma=len(lis)/50000   # std of the gaussian filter (fudge factor)
    lisgaus=[gauss(k/100-5*sigma,sigma) for k in range(int(1000*sigma))] #Gaussian List
    lis2=scipy.signal.convolve(lis,lisgaus) # Convolved list
    off=int((len(lisgaus)-1)/2) # Shift of the convolved list
    l3=[lis2[k+off] for k in range(len(lis2)-2*off)]  # Smoothed list (offset taken into account)
    p=list(scipy.signal.find_peaks(l3)[0]) # Up Peaks List
    p2=scipy.signal.find_peaks(-1*np.array(l3))[0] # Down Peaks List
    p.extend(p2) # Combination of both lists
    p.sort()
    return(p)  # Total Peaks

In [ ]:
# lis: Float list, Triangle signal to be processed
# This Function smooth the triangle to erase noise (like previous function)
# Exact same process as the previous function but return the smoothen triangle rather than the peaks

def smooth_triangle(lis):
    sigma=len(lis)/50000
    lisgaus=[gauss(k/100-5*sigma,sigma) for k in range(int(1000*sigma))]
    lis2=scipy.signal.convolve(lis,lisgaus)
    off=int((len(lisgaus)-1)/2)
    l3=np.array([lis2[k+off] for k in range(len(lis2)-2*off)])
    l3*=np.mean(lis)/np.mean(l3)
    return(l3)

In [ ]:
# lis : Float lis to be filtered
# filt : Float used to filter the list
# This function try to erase noise from a signal. It get a list lis and remove all the elements that are lower than
# "mean + filtr*sqrt(var)". All the values of those elements are set to the mean.
# We return the filtered list

def filtr(lis,filt):
    l2=deepcopy(lis)
    m=np.mean(l2)
    v=np.sqrt(np.var(l2))
    for k in range(len(l2)):
        if l2[k]<m+filt*v: # Conditiion to erase noise
            l2[k]=m #Set noise to the mean
    return (l2)

In [ ]:
# lis : Float lis to be filtered
# filt : Float used to filter the list
# This function filter the list with the previous function and then use a gaussian filter 
# to convolve the signal and smooth it.
# We return an array of the peaks of l3

def Smooth(lis,filt):
    sigma=len(lis)/500000 # std of the gaussian filter (fudge factor)
    lisgaus=[gauss(k/100-5*sigma,sigma) for k in range(int(1000*sigma))] # Gaussian List
    lis2=filtr(lis,filt) # Filtered list
    lis2=scipy.signal.convolve(lis2,lisgaus)  # Convolved list
    off=int((len(lisgaus)-1)/2) # Offset of convolution
    l3=[lis2[k+off] for k in range(len(lis2)-2*off)] # Smoothed list (offset taken into account)
    return(l3)

### Detect peaks from smoothed triangular wave

In [ ]:
# lis : Float list, orginal signal 
# l3 : Float list, result of the previous function used with lis
# This function get the peaks of lis. First we get a region for those peaks based on the peaks of the smoothed peaks l3
# Then we check the maxima in thoses region and get the peaks from that.
# We return an array of all the peaks 


def PeaksfromSmoothed(lis,l3):
    m=min(l3[int(len(l3)/4):int(3*len(l3)/4)]) # Minimum of the smoothed list l3 (without taking account of artifact from convolution)
    p=scipy.signal.find_peaks(l3)[0] # Peaks of l3
    zones=[] # list containing different maxima zones
    for j in p:
        if l3[j]-m>0: # Check if the peaks are above the mean, if not, it is not a good peak
            z1=0
            z2=len(l3)
            for k in range(j): 
                if l3[j-k]-m<(l3[j]-m)/2: # Create a zone at the left middle of the gaussian
                    z1=j-k
                    break  
            for k in range(len(l3)-j):
                if l3[j+k]-m<(l3[j]-m)/2:  # Create a zone at the right middle of the gaussian
                    z2=j+k
                    break
            if z1!=0 and z2!=len(l3): # Check if we found the middle of gaussian, else, this was not a peak but a edge
                zones.append([z1,z2]) 
    Maxes=[]
    for k in zones:
        z1,z2=k
        Maxes.append(np.argmax(lis[z1:z2])+z1) #Take the maxima of lis in each zone as the peak 
    return(Maxes)

In [ ]:
# peaks : Float list, list of the differents peaks of the main signal
# separator : Float list, list of the peaks of the triangle signal
# This function will separate the peaks according to the triangle sweep.
# It will retrun a 2D array, the first dimension being the number of the sweep 
# we are looking at and the second being the peaks for this sweep


def SeparatePeaks(peaks,separator):
    PeaksSeparated=[[] for  k in range(len(separator)+1)] # 2D Array that will contains the list 
    for k in range(len(peaks)):
        so=list(separator.copy())
        so.append(peaks[k])
        so.sort() # We check where the peaks is compared to the peaks of the triangle signal
        i=so.index(peaks[k])
        PeaksSeparated[i].append(peaks[k]) # We append the list at the number of the sweep
    return(PeaksSeparated)

In [ ]:
# lis : Float list, signal to find peak of
# fg : Float list, this is the function generator signal, equivalent to separator in the previous function
# filt : Float, filter to apply to lis, same as in the function "filtr"
# trace : Bool, true if you want to trace the signal + the peaks found by the programm
# It will return a 2D array, the first dimension being the number of the sweep 
# we are looking at and the second being the peaks for this sweep
# This function use all the previous function to find the peaks of a signal with a set factor to filter filt


def FindPeak (lis,fg,filt=3,trace=False):
    l3=Smooth(lis,filt)  # Erase Noise
    Maxes=PeaksfromSmoothed(lis,l3) # Get Peaks
    Separation=ProcessTriangle(fg)[:-1] #Get the smoothed triangle signal
    PeaksSeparated=SeparatePeaks(Maxes,Separation) #Separate the peaks according to the sweep
    if trace:  # This subsection just trace the plot if you want to see where the peaks are
        m=np.mean(lis)
        Ml=[m for k in lis]
        for k in Maxes:
            Ml[k]=lis[k]
        plt.figure()
        plt.plot(lis)
        plt.plot(Ml)
    for k in range(len(PeaksSeparated)): # This subsection remove the duplicates
        new_list = [] 
        for i in PeaksSeparated[k] : 
            if i not in new_list:  # Test wether the same peaks are counted twice
                new_list.append(i)
        PeaksSeparated[k]=new_list
    return(PeaksSeparated)

In [ ]:
# lis : Float list, signal to find peak of
# fg : Float list, this is the function generator signal
# It will return a 2D array, the first dimension being the number of the sweep 
# we are looking at the second being the peaks for this sweep
# This function is the final function we use for peak finding. Contrary to the previous one, we do not set manually the 
# factor for filtering the signal. It is chosen to be the one for which we have at least 60% of the sweeps that have 
# the same number of detected modes
# This is to assure that the the factor is choosed in a good spot for which most parasitic sweeps are removed but we still 
# got the modes we are looking for.
# It is really important to have the same number in each sweep to get the statisitic of each height.
# Furthermore, we make a difference between the increasing and decreasing sweeps. At the end, we chosse to conserve only
# one of the two based on the number of modes detected and the proportion of sweeps with the same number of modes.

def DefinitiveFindPeak(lis,fg):
    
    def toOpt(m): # function that calculate the proportion of sweeps with the same number of mode and this number for an input filter factor m
        p=FindPeak(lis,fg,filt=m,trace=False) # Find Peak with a filter m
        taille=[len(p[2*k+1]) for k in range(int(len(p)/2))] # This function do that for the increasing sweeps
        taille=taille[1:]
        a,b=Counter(taille).most_common(1)[0] # Calculate the number of sweeps with the most common number of modes detected and that said number
        return(b/len(taille),a) # return the proportion and the number of modes

    def toOpt2(m): # Do the exact same thing as "toopt" but for the decreasing sweeps
        p=FindPeak(lis,fg,filt=m,trace=False)
        taille=[len(p[2*k]) for k in range(int(len(p)/2))]
        taille=taille[1:]
        a,b=Counter(taille).most_common(1)[0]
        return(b/len(taille),a)
    
    # Calculate the minimal filter for which at least one mode is detected and 60% (fudge factor) of the sweeps have the same number of modes
    j=0
    a_odd=1
    b_odd=0
    while a_odd>0 and b_odd<0.6:
        b_odd,a_odd=toOpt(j/5) # Done for increasing sweeps
        j+=1
    filt_odd=(j-1)/5 # Get the filter factor for which this condition is achieved
    
     # Calculate the minimal filter for which at least one mode is detected and 60% (fudge factor) of the sweeps have the same number of modes
    j=0
    a_even=1
    b_even=0
    while a_even>0 and b_even<0.6:
        b_even,a_even=toOpt2(j/5) # Done for decreasing sweeps
        j+=1
    filt_even=(j-1)/5 # Get the filter factor for which this condition is achieved
    
    
    # This subsection decide whether we should consider the increasing or the decreasing sweeps
    decision=filt_odd>filt_even #if the filter is lower in one case we choose that because that means that the signal is less noisy
    if filt_even==filt_odd: 
        decision=b_odd<b_even # if they are the same, we choose the highest proportion
        if b_odd==b_even:
            decision=a_odd<a_even # if they are the same, we choose the higher number of detected modes
            
    if decision:
        peaks=FindPeak(lis,fg,filt=filt_even,trace=False) #choose the good filter factor
        peaks=[peaks[2*k] for k in range(1,int(len(peaks)/2))] # Return the peaks for the increasing sweeps
    else:
        peaks=FindPeak(lis,fg,filt=filt_odd,trace=False) #choose the good filter factor
        peaks=[peaks[2*k+1] for k in range(1,int(len(peaks)/2))] # Return the peaks for the decreasing sweeps
        
    return(peaks)   

In [ ]:
# lis : Float list, signal to find peak of
# fg : Float list, this is the function generator signal
# green : Float list, this is the green parasitic signal
# Return the evolution of the height of each peak, the mean of the triangle voltage associated to the peaks
# the mean of the green signal for this voltage
# and the index telling which sweeps have the good number of modes detected.

def HeightPerModes(lis,fg,green):
    PS=DefinitiveFindPeak(lis,fg) # Find the peaks of lis
    PS=PS[1:-1]
    idx=[]
    t=Counter([len(p) for p in PS ]).most_common(1)[0][0] # Most common number of modes found during the sweep
    for p in PS:
        if len(p)==t:
            idx.append(p)     # Keep only the sweeps for which the same number of modes was detected
    HoM=[[lis[p[l]] for p in idx] for l in range(len(idx[0]))] # Height evolution of each modes
    VoM=[np.mean([smooth_triangle(fg)[p[l]] for p in idx]) for l in range(len(idx[0]))] # Mean of the triangle voltage for each mode
    HoG=[np.mean([green[p[l]] for p in idx]) for l in range(len(idx[0]))] # Mean of the height of the green for each peaks
    return(HoM,VoM,HoG,idx)

In [ ]:
# H : Float list, Height evolution of each mode
# This function return the variance of each mode divided by its mean

def StatHeight(H):
    return([np.sqrt(np.var(h))/np.mean(h) for h in H]) # variance of the height of each modes divided by its mean

# Converting OPO signal to bitstream

Once the proper opo mode is found, the opo signal passes the homodyne system and reaches at the photodiode. This section explains the code that is used to 1) convert opo signal to bitstream and 2) measure the probability of the bitstream and do simple statistical analysis to determine whether the bitstream is random or not.

In [ ]:
# movmean function, get the mean value of lis for given range of N
def movmean(lis,N):
    lis2=np.zeros_like(lis)
    for k in range(len(lis)):
        lis2[k]=np.mean(lis[max(int(k-(N-1)/2),0):min(int(k+(N+1)/2),len(lis))]) 
    return lis2

In [ ]:
# Principal function to read the OPO signal and convert it the bitstream of 0 and 1
# t : Float list, time signal from the oscilloscope
# sig : Float list, OPO signal from the oscilloscope
# mod : Float list, Modulation signal
# Return the bit stream and if the sequence passes the naive test

def BitStream(t, sig, mod, draw_flag, modulation_up_through_flag = False, moving_mean_flag = False, contrast_adjust = True):
    ''' modulation_up_through_flag is True when the max of the modulation corresponds to the laser signal going through '''
    if not(modulation_up_through_flag): #whether the positive edge of the modulation signal builds up the opo signal or not.
        mod = - mod
    ndata = len(t)
    
    ##simple fft and filtering the signal. Variables such as filter_order and MinPeakDistane may change depending on the quality of the signal
    T = t[2]-t[1]
    Fs = 1/T
    f = np.linspace(-Fs/2,Fs/2,num=ndata)
    fftvec = np.abs(np.fft.fftshift(np.fft.fft(sig-np.mean(sig))))
    ModFreq = abs(f[np.argmax(fftvec)])
    MinPeakDistance0 = int(1/ModFreq/T)
    MinPeakDistance = 0.75*MinPeakDistance0
    filter_order = 5
    if int(MinPeakDistance/10)%2==0:
        window_length = max(int(MinPeakDistance/10) +1, filter_order+2)
    else:
        window_length = max(int(MinPeakDistance/10), filter_order+2)
    y=scipy.signal.savgol_filter(sig,window_length,filter_order)
    ymod=scipy.signal.savgol_filter(mod,window_length,filter_order)
    
    #find min max value of modulation signal. Peak prominence may change depending on the quality of the signal
    max_value = np.max(np.diff(ymod[int(ndata/10):9*int(ndata/10)]))
    min_value = np.min(np.diff(ymod[int(ndata/10):9*int(ndata/10)]))
    peak_prominence = 0.4
    mod_up_t = scipy.signal.find_peaks(np.diff(ymod), height=peak_prominence * max_value, distance=MinPeakDistance)[0]
    mod_down_t = scipy.signal.find_peaks(np.diff(-ymod), height=-peak_prominence * min_value, distance=MinPeakDistance)[0]
    offset = int(np.mean(np.diff(mod_up_t))/10);
    if mod_up_t[0] > mod_down_t[0]:
        order = 1
    else:
        order = 0
    mm,modu,modu_std,xx=[0 for k in range(min(len(mod_up_t), len(mod_down_t))-1)],[0 for k in range(min(len(mod_up_t), len(mod_down_t))-1)],[0 for k in range(min(len(mod_up_t), len(mod_down_t))-1)],[0 for k in range(min(len(mod_up_t), len(mod_down_t))-1)]
    for i in range(min(len(mod_up_t), len(mod_down_t))-1):
        if mod_up_t[i] < mod_down_t[i+order]:
            mm[i] = np.mean(y[mod_up_t[i]+offset:mod_down_t[i+order]-offset])
            modu[i] = np.mean(mod[mod_up_t[i]:mod_down_t[i+order]])
            modu_std[i] = np.std(mod[mod_up_t[i]:mod_down_t[i+order]])      
            xx[i] = t[int(np.floor((mod_up_t[i] + mod_down_t[i+order])/2))]
    
    #if moving_mean_flag is false, we plot a straight line of threshold with a bitstream.
    if not(moving_mean_flag):
        threshold = [(min(mm)+max(mm))/2]*len(mm)
    else:
        threshold = movmean(mm, 50)
       
    #once the threshold is determined, we sort the peak of the signal to either high (bit 1) or low (bit 0).
    low, high, mm0, xx0, mm1, xx1 = [], [], [], [], [], []
    for k in range(len(mm)):
        if mm[k]<threshold[k]:
            low.append(k)
            mm0.append(mm[k])
            xx0.append(xx[k])
        if mm[k]>threshold[k]:
            high.append(k)
            mm1.append(mm[k])
            xx1.append(xx[k])

    bit_stream = np.in1d(np.sort(low+high),high)
    nbits = len(bit_stream)
    mean_bit = np.mean(bit_stream) #basically, it will give the probability, since bit_stream is either 1 or 0.
    bit_flip_prob = np.mean(abs(np.diff(bit_stream))) #the easiest way of checking the true randomness at the first stage is to check whether the bit is flipping randomly (0.5)
    test_pass = abs(mean_bit-0.5)<1/np.sqrt(nbits) and abs(bit_flip_prob-0.5)<1/sqrt(nbits)
    
    #set draw_flag to false if you don't want to plot and just get the bitstream result.
    if draw_flag==True:
        print('Extracted repetition rate = '+ str(1/(t[MinPeakDistance0]-t[1]))+'Hz \n')
        print('Number of Bits %s \n' %nbits)
        print('Bit Mean %s \n' %mean_bit)
        print('Bit Flip Probability %s \n' %bit_flip_prob)

    print('Passes simple RNG test (mean and flip)? %s (%s, %s) \n' %(test_pass, mean_bit, bit_flip_prob) )
   
    bit_str_plt=[]
    up_plt=max(sig)
    down_plt=min(sig)
    for k in range(len(bit_stream)):
        if bit_stream[k]:
             bit_str_plt+=[up_plt]*(mod_up_t[k+1]-mod_up_t[k])
        else:
            bit_str_plt+=[down_plt]*(mod_up_t[k+1]-mod_up_t[k])
    if draw_flag==True:
        plt.figure()
        plt.plot(t, sig - np.min(sig), color = 'black')
    #     plt.plot(t, (mod - np.min(mod))/(np.max(mod) - np.min(mod))*np.mean(sig), color = 'grey')
    #     plt.plot(np.array(bit_str_plt),alpha=0.5)
        plt.bar(xx0, mm0, width = 0.00005, alpha = 0.5, color = 'red')
        plt.bar(xx1, mm1, width = 0.00005, alpha = 0.5, color = 'blue')
        if moving_mean_flag:
            plt.plot(xx, threshold - np.min(sig), linestyle = '--')
        else:       
            plt.plot(t, [threshold[0] for k in sig] - np.min(sig))
    #return(bit_stream,test_pass)
    if contrast_adjust == True:
        return(bit_stream,test_pass,mm1,mm0)
    else:
        return(bit_stream,test_pass)
        

# Workflow

Put all the functions previously defined. 

### OPO setup

In [ ]:
# First function to use, connect all the component and set them to their default behavior
# The default behavior is the sweep of the cavity
# Input nothing and return nothing
# Define global variable m,o,p,f,r,pm so be carefull not to use them

def Setup():
    global m,o,p,f,r,pm
    m,o,p,f,r,pm=ConnectAll()
    f.stop(func_channel=1)
    f.stop(func_channel=2)
    p.stop()
    f.set_function_ramp(func_channel=func_channel,sym=func_symmetry,upV=volt_high,lowV=volt_low,phase=phase,freq=freq)
    f.start(func_channel=func_channel)
    o.set_time(2/freq/10)
    o.conn.WriteString("TRig_MoDe AUTO",True)
    o.conn.writeString("C"+str(opo_ch_source)+":CPL D50",True)
    o.conn.writeString("C"+str(error_ch_source)+":CPL D50",True)
    o.conn.writeString("C"+str(volt_ch_source)+":CPL D1M",True)
    o.conn.writeString("C"+str(modulation_ch_source)+":CPL D1M",True)
    p.stop()
    p.set_offset(offset)
    p.setup_control(offset=offset,p_gain=p_gain,i_gain=i_gain)
    p.conn.query("GETN? 1, 120")

In [ ]:
# Exact same function as before but do not connect the components
# Use this function if you want to reset the electronics, but do want to connect the components again.
def Setup_Fast():
    f.stop(func_channel=1)
    f.stop(func_channel=2)
    p.stop()
    f.set_function_ramp(func_channel=func_channel,sym=func_symmetry,upV=volt_high,lowV=volt_low,phase=phase,freq=freq)
    f.start(func_channel=func_channel)
    o.set_time(2/freq/10)
    o.conn.WriteString("TRig_MoDe AUTO",True)
    o.conn.writeString("C"+str(opo_ch_source)+":CPL D50",True)
    o.conn.writeString("C"+str(error_ch_source)+":CPL D50",True)
    o.conn.writeString("C"+str(volt_ch_source)+":CPL D1M",True)
    o.conn.writeString("C"+str(modulation_ch_source)+":CPL D1M",True)

In [ ]:
# Do an entire Mode sweep of the cavity, get every voltage for each mode and their height evolution

def Mode_Sweep():
    f.stop(func_channel=2)
    p.stop()
    f.set_function_ramp(func_channel=func_channel,sym=func_symmetry,upV=volt_high,lowV=volt_low,phase=phase,freq=freq)
    f.start() # Start triangle sweep
    o.set_time(20/freq/10) # Get 20 sweeps
    time.sleep(2)
    o.conn.WriteString("STOP",True) # Stop the oscillo before acquisition
    t,a1=o.acquire(opo_ch_source)
    t2,a2=o.acquire(volt_ch_source)
    t3,a3=o.acquire(error_ch_source) # acquire all OPO signals
    Height,Voltage,GreenHeight,Peaks=HeightPerModes(a1,a2,a3) # get the height evolution of each mode
    o.conn.WriteString("TRig_MoDe AUTO",True) # reset the oscillo trigger
    Var=StatHeight(Height) # get the variance in the height of each mode
    return (t,a1,a2,a3,Peaks,Height,Voltage,GreenHeight,Var) # return everything to be used after

In [ ]:
# v : Float, theoritical piezo voltage the mode is supposed to bet at after doing the initial sweep
# It returns the real good voltage one shouls be using to lock (Float)
# This function does a little piezo DC voltage sweep near the theoritical position
# This is because piezo does not respond the same to a triangle and to a DC voltage therefore a little adjustement is needed
# We then take the voltage of the highest peak in this sweep

def GoodVoltage(v):
    p.stop() #stop PID
    sig=[]
    vol=[]
    scale=(volt_high-volt_low)/10 # range of the DC sweep (arbitrary factor)
    for k in range(100):
        volt=v-scale/2+scale*k/100
        f.set_function_DC(volt,func_channel=func_channel) # send a DC voltage
        time.sleep(0.1)
        t,a=o.acquire(opo_ch_source) # get the OPO signal for this voltage
        t,b=o.acquire(volt_ch_source) # get the voltage signal (real one)
        sig.append(np.mean(a))
        vol.append(np.mean(b))
    plt.figure()
    plt.plot(vol,sig) # Plot the result of the sweep to help the user
    plt.xlabel("DC voltage send to the piezo (V)")
    plt.ylabel("Mean of the OPO signal (V)")
    plt.title("Result of the DC sweep near the value found with the triangle sweep") 
    goodv=vol[np.argmax(sig)] # the voltage choosen is the highest peak in the sweep
    return(goodv)

In [ ]:
# This function does the setpoint sweep
# It varies the setpoint and then choose the setpoint for which mean/std is the highest
# It then return the setpoint (Float)

def GoodSetpoint():
    sig,var=[],[]
    stp=[]
    k=0
    p.stop() # stop the piezo
    p.control(0) # enable piezo with setpoint 0
    p.stop()
    time.sleep(1)
    p.get_out()
    time.sleep(1)
    p.stop()
    p.control(0)
    p.stop()
    time.sleep(1)
    p.get_out()
    time.sleep(1) 
    # All the previous are initizalization calls to be sure the setpoint send us the good output, 
    # indeed sometime it has some buses in memory. This is done to get those buses and also to set it to setpoint=0
    while (min_out<p.get_out()<max_out or k==0): # we are sweeping setpoint while the PID can send sufficient voltage
        stp.append(k/100) # setpoint used
        p.stop()
        time.sleep(0.1)
        p.control(k/100) # control with the setpoint
        time.sleep(0.5)
        t,a=o.acquire(opo_ch_source) # acquire the signal
        sig.append(np.mean(a)) # get mean
        var.append(np.sqrt(np.var(a))) #get std
        k+=1
    p.stop()
    plt.figure()
    plt.plot(stp,sig)
    plt.plot(stp,var) # plot the sweep result to help the user
    plt.xlabel("Setpoint used")
    plt.ylabel("Voltage (V)")
    plt.legend(["Mean of the signal (V)","Std of the signal (V)"])
    plt.title("Result of the Setpoint Sweep")
    goods=stp[np.argmax(np.array(sig-np.min(sig))/var)] # Take the setpoint with better ratio mean/std
    return(goods)

### Turn on Modulation signal

In [29]:
# s : Float, Setpoint used without modulation
# fudge, Float, how the user thinks the setpoint will change when enabling modulation
# This function enable modulation and change the setpoint

def Modulation(s,fudge=1/2.):
    f.set_function_square(func_channel=modulation_channel,sym=modulation_simmetry,phase=modulation_phase,upV=modulation_volt_high,lowV=modulation_volt_low,freq=modulation_freq)
    f.start(modulation_channel) # enable modulation
    o.set_time(5/modulation_freq) # get good time acquisition (50 bits)
    p.control(s*fudge) # change the setpoint
    return()

### Lock to a specific OPO mode

In [ ]:
# This function Lock the cavity on the highest variance mode (without modulation)
# The input lock is a string flag telling which mode you want to lock in
# "v" correspond to the highest variance mode
# "h" : correspond to the highest mode
# If the string is an int, you will lock on the n'th mode
# The general idea is to do the mode sweep with triangle
# Then the smaller DC sweep near the good voltage
# Then the setpoint sweep
# It return the DC voltage used and the setpoint used
# It also plot the results of DC sweep and setpoint sweep to help the user undertsand what went wrong if the locking didn't happen

def Locking(lock="v"):
    Setup_Fast()  # Initialize the cavity to sweeping
    t,OPO,Volt,Green,Peaks,Height,Voltage,GreenHeight,Var=Mode_Sweep() # Get all the mode sweeping data
    o.set_time(2/1000) # Zoom in 
    print("Initial Sweep Done")
    H=[np.mean(h) for h in Height] # Height list of the modes
    print("Number of good modes detected : " + str(len(H)))
    if lock=="v":
        v=Voltage[np.argmax(Var)] # Voltage of the mode with highest variance 
    elif lock=="h":
         v=Voltage[np.argmax(H)] # Voltage of the mode with highest intensity 
    elif lock.isdigit():
        v=Voltage[int(lock)] # Voltage of the lock'th mode
    else:
        raise ValueError('Input is not a correct flag')
    print( "Initial Voltage guess : "+str(v)+" V")
    v=GoodVoltage(v) # DC voltage sweep
    f.set_function_DC(v,func_channel=func_channel) # Set DC voltage
    print("DC Sweep done. Voltage Found at : " + str(v)+" V")
    s=GoodSetpoint() # Setpoint sweep
    for k in range(int(s*100+1)): # We once again rise the setpoint to the good setpoint s
        p.stop()
        time.sleep(0.1)
        p.control(k/100) # rise setpoint
        time.sleep(0.5)
    print("Setpoint Sweep done. Setpoint used : " + str(s) + " V")
    return(v,s) # return voltage/setpoint

### Move piezo stage to give time delay between bias and pump

In our manuscript, we introduce a simple reconstruction of weak bias field. We give time delay between bias and pump by changing the optical path of bias line with piezo stage and measure the probability. The code below introduces how we can control the piezo and measure the probability.

In [ ]:
# Npoints : Int, number of points you want to acquire
# path : str, path in which you want to save the datas (General file, the programm will create subfolder by itself)
# This programm does the piezo sweep, it will acquire bit datas then move the piezo and so on...
# All the datas will be saved in "Path/%Day/P-Sweep/%Number of P-sweep of the day"
# It returns the piezo position vector and the probability for each position vector


def Piezo_Sweep(NPoints, step_size, pm_start_position=0, path=r'your_own_path',draw_flag=False):
    pmbis=pm # By default, this is supposed to be the piezo motor
    try:
        pmbis=pylablib.devices.Thorlabs.kinesis.KinesisPiezoMotor("your_own_device_number") # if not connected, connecte to the piezo motor
        # if the device is still not connected, try to unplug and plug again
    except KeyboardInterrupt: # if user interrupt we stop the loop
        pass        
    finally: # All this subsection just create the folder in which one saves the data
        today = date.today()
        today=str(today)
        try:
            os.mkdir(path+"/"+today)
        except FileExistsError:
            pass
        try:
            os.mkdir(path+"/"+today+"/P-Sweep")
        except FileExistsError:
            pass
        nbr=len([name for name in os.listdir(path+"/"+today+"/P-Sweep")])
        path=path+"/"+today+"/P-Sweep/Sweep N°"+str(nbr+1)
        os.mkdir(path)

        Pos=[pm_start_position] # Position of the piezo list
        Prob=[] # Probability list
        move_sign=1
        moving_unit=2
        contrast_target = 0.08 
        #sometimes, the opo bitstream can lose its contrast gradually so the code below is a simple code to 
        #adjust the contrast by moving one of the arm of homodyne detection with piezo stage. 
        previous_contrast= 0
        check_flag=True
        for k in range(NPoints):
            print('Taking point n{0}'.format(k))
            #Data, b, tp = Take_temp(draw_flag) # Take the datas
            Data, b, tp, mm1, mm0 = Take_temp(draw_flag)
            # Pseudo code for contrast optimization
            current_contrast= (mm1[np.argmax(mm1)]-mm0[np.argmin(mm0)])/(mm1[np.argmax(mm1)]+mm0[np.argmin(mm0)])
            if False :
                if current_contrast < contrast_target:
                    if current_contrast < previous_contrast:
                        move_sign=-move_sign
                        pmbis.move_by(2*moving_unit*move_sign, channel=2)
                        pmbis.wait_move(channel=2)
                        previous_contrast=current_contrast
                    elif current_contrast > previous_contrast:
                        pmbis.move_by(moving_unit*move_sign, channel=2)
                        pmbis.wait_move(channel=2)
                        previous_contrast=current_contrast
                else:
                    previous_contrast= 0
            time.sleep(1)
            Prob.append(np.mean(b)) # this is the probability of 1
            np.save(path+"/Data_at_pos_"+str(Pos[-1])+".npy",Data) # save datas
            np.save(path+"/Data_at_pos_pros"+str(Pos[-1])+".npy",(Pos[-1],Prob[-1]))
            pmbis.move_by(step_size, channel=1) # move piezo by 1 unit (can change)
            pmbis.wait_move(channel=1) # wait till move end
            Pos.append(Pos[-1]+step_size) # Update Pos
            if (Prob[-1] < 0.1) or (Prob[-1]> 0.9): # We move the piezo further by 1 if prob are nearly 0 or 1 (can change the conditions)
                pmbis.move_by(step_size, channel=1)
                pmbis.wait_move(channel=1)
                Pos[-1]+=step_size
        Pos=Pos[:-1] # Remove last position as it was not used
        plt.figure()
        plt.plot(Pos,Prob)
        plt.xlabel("Piezo Step")
        plt.ylabel("Probability of 1")
        plt.title("Probability tunability with the time delay")  # Create the final plot position vs probability and then save it
        plt.savefig(path+"/Result.png")
        np.save(path+"/Result.npy",[Pos,Prob])
        return(Pos,Prob) # Return both the position and probability vector for further anlysis/plot

### Controlling the waveplate in front of the bias

In our manuscript, we rotate the waveplate in front of the bias to tune the power of the bias and check how probability changes based on the rotation angle. The code below is the function used to rotate the waveplate and measure the probability of the bitstream.

In [31]:
# Npoints : Int, number of points you want to acquire
# path : str, path in which ou want to save the datas (General file, the programm will create subfolder by itself)
# This programm does the waveplate sweep, it will acquire bit datas then move the waveplate by one degree and so on...
# All the datas will be saved in "Path/%Day/W-Sweep/%Number of W-sweep of the day"
# It returns the waveplate angles vector and the probability for each position vector


def Waveplate_Sweep(NPoints, unit_rotate, path=r'your_own_path',draw_flag=False):
    # All this subsection just create the folder in which one save the dataas
    today = date.today()
    today=str(today)
    try:
        os.mkdir(path+"/"+today)
    except FileExistsError:
        pass
    try:
        os.mkdir(path+"/"+today+"/W-Sweep")
    except FileExistsError:
        pass
    nbr=len([name for name in os.listdir(path+"/"+today+"/W-Sweep")])
    path=path+"/"+today+"/W-Sweep/Sweep N°"+str(nbr+1)
    os.mkdir(path)

    Pos=[0] # Angle of the waveplate list
    r.moveto(-32+0) ##
    Prob=[] # Probability list
    for k in range(NPoints):
        time.sleep(3)
        Data,b,tp=Take_temp(draw_flag) # Take the datas
        Prob.append(np.mean(b)) # this is the probability of "1"
        np.save(path+"/Data_at_angle_"+str(Pos[-1])+".npy", Data) # save datas
        r.moveby(unit_rotate) # move wavplate by 1 degree (can change the 1) 
        Pos.append(Pos[-1]+unit_rotate) # Update Pos
    Pos=Pos[:-1] # Remove last position as it was not used
    plt.figure()
    plt.plot(Pos,Prob)
    plt.xlabel("Waveplate Angle")
    plt.ylabel("Probability of 1")
    plt.title("Probability tunability with the waveplate angle")  # Create the final plot angle vs probability and then save it
    plt.savefig(path+"/Result.png")
    np.save(path+"/Result.npy",[Pos,Prob])
    return(Pos,Prob) # Return both the position and probability vector for further anlysis/plot

### Acquire bitstreams and save the data

Based on the previous "Bistream" function, the following functions include options to change the number of bits to measure at one measurement, save the data in the local machine, etc.

In [32]:
# N : Int, Number of bits you want to acquire
# This function changes the time acquisition of the oscilloscope and ask the user to change the number of points

def Bit_Number(N):
    T=N/modulation_freq/10
    o.set_time(T) # Zoom to have the correct number  of bits displayed
    N_points=T/(2/1000)*50000 # Number of points one should have
    print("Set the number of points of the oscilloscope to " + str(N_points)) # Ask the user

In [ ]:
# This function justtake oscilloscope datas and converts them into a bit stream
# It returns the datas (Float Array x3), the bistream (Bool Array) and a boolean indicating if the naive test is passed 
# The datas are in order : [[time, signal, modulation],bit stream, boolean]

def Take_temp(draw_flag):
    o.conn.WriteString("STOP",True) # Stop the oscillo to acquire
    t,sig=o.acquire(opo_ch_source) # acquire OPO signal
    t,mod=o.acquire(modulation_ch_source) # acquire modulation signal
    o.conn.WriteString("TRig_MoDe AUTO",True) # Restart the oscillo
    #b,tp=BitStream(t,sig,mod,draw_flag) # Get the bistream out of datas
    b,tp,mm1,mm0=BitStream(t,sig,mod,draw_flag) # Get the bistream out of datas
    return([t,sig,mod],b,tp,mm1,mm0) #return all values

In [ ]:
# path : str, path in which ou want to save the datas (General file, the programm will create subfolder by itself)
# This function take datas and then save them in "path/%Date/Good-Bad/Data_Numberofdatas_NumberOfBits".
# The Good/Bad folder is chosen depending on if the datas are considered good with the naive test or bad
# datas are saved following : [[time, OPo signal, Modulation signal], bit stream] 

def Take(path=r'your_own_path'):
    draw_flag=True
    signal,bit_stream,test_pass=Take_temp(draw_flag) # acquire all datas
    today = date.today()
    today=str(today)
    try:
        os.mkdir(path+"/"+today)
    except FileExistsError:
        pass
    if test_pass:
        new_path=path+"/"+today+"/Good"
    else:
        new_path=path+"/"+today+"/Bad"
    try:
        os.mkdir(new_path)
    except FileExistsError:
        pass # All the previous section was to create the folder if needed
    nbr=len([name for name in os.listdir(new_path)])
    np.save(new_path+"/Data_"+str(nbr+1)+"_"+str(len(bit_stream))+"bits"+".npy",[signal,bit_stream]) # save datas
    return(bit_stream) # return the bit stream

In [35]:
#N is the number of bits we want to look at
#modulatoin frequency set to 1000
#we want to set sampling rate as constant. Now it is 1000kS/s

def Take_Nbits(N, draw_flag, sampling_rate, path=r'your_own_path', modulation_freq=10000):

    time_string_1="TDIV "
    time_string_2=" S"
    memory_str="MSIZ "
    total_time=N/modulation_freq
    set_time_division_num=total_time/10
    set_time_division_str=str(set_time_division_num)
    set_MSIZ_num=total_time*sampling_rate
    temp=time_string_1+set_time_division_str+time_string_2
    o.conn.WriteString(temp,True) #put time division to the oscillo 
    temp=memory_str+str(set_MSIZ_num)
    o.conn.WriteString(temp,True) #put memory size to the oscillo

    #Oscilloscope only allows discrete values of memory size, so certain discrete values of bit numbers are allowed.
    #Therefore, we have to check the current memory size and time division of oscilloscope.
    
    o.conn.WriteString("Memory_SIZe?", True)
    current_memory_str=o.conn.ReadString(500) # get current memory size as a string
    #print(current_memory_str)
    if (current_memory_str.__contains__("MSIZ")):
        txt=current_memory_str.split()[1]
    else:
        txt=current_memory_str
    o.conn.WriteString("Time_DIV?", True) 
    current_time_div_string=o.conn.ReadString(500) # get current time division as a string    
    if (current_time_div_string.__contains__("TDIV")):
        txt2=current_time_div_string.split()[1]
    else:
        txt2=current_time_div_string  
    #print(txt2)
    current_time_div_num=float(txt2)    
    
    current_memory_num=0
    emp_str=""
    for m in txt:
        if m.isdigit():
            emp_str+=m #get only numbers, not letters
    if txt != "2.5M": #we separate the case of 2.5M, since this is the only case that includes "point" in the string
        if txt.find("K")!=-1: #returns -1 when there is no specific letter
            current_memory_num=int(float(emp_str))*1000
        elif txt.find("M")!=-1:
            current_memory_num=int(float(emp_str))*1000000
        else:
            current_memory_num=int(float(emp_str))
    else:
        current_memory_num=2500000    
    memory_array_str = ["500", "1000", "2500", "5000", "10K", "25K", "50K", "100K", "250K", "500K", "1M", "2.5M", "5M", "10M", "25M"]
    memory_array_num = [500, 1000, 2500, 5000, 10000, 25000, 50000, 100000, 250000, 500000, 1000000, 2500000,5000000, 10000000, 25000000]
    
    current_memory_str=""
   
    while current_memory_num < sampling_rate*current_time_div_num*10:
            index=memory_array_num.index(current_memory_num)
            index+=1
            current_memory_num=memory_array_num[index]
            current_memory_str=memory_array_str[index] #we get the desirable memory size which doesn't reduce the sampling rate
    
    temp=memory_str+current_memory_str
    o.conn.WriteString(temp,True) 
    temp=time_string_1+current_time_div_string+time_string_2
    o.conn.WriteString(temp,True) 
    time.sleep(2*total_time) # this time, need to be computed based on the number of bit. E.g. 10k 10kHz need 1s, 100k 10kHz need 10s
    signal,bit_stream,test_pass,mm1,mm0=Take_temp(draw_flag)
    today = date.today()
    today=str(today)
    try:
        os.mkdir(path+"/"+today)
    except FileExistsError:
        pass
    if test_pass:
        new_path=path+"/"+today+"/Good"
    else:
        new_path=path+"/"+today+"/Bad"
    try:
        os.mkdir(new_path)
    except FileExistsError:
        pass # All the previous section was to create the folder if needed
    nbr=len([name for name in os.listdir(new_path)])
    np.save(new_path+"/Data_"+str(nbr+1)+"_"+str(len(bit_stream))+"bits"+".npy",[signal,bit_stream]) # save datas
    
    
    return(bit_stream)  #return all values

# Sandbox

### Initialize the OPO setup

In [11]:
Setup()
#Setup_Fast()

### Lock to a specific OPO mode

The code below tells you which voltage and setpoint to use to lock the mode. In case it doesn't work because the OPO signal is not strong or noisy, you can skip the code and find the mode manually by looking at the oscilloscope  

In [ ]:
v, s = Locking()

### Turn on modulation

In [37]:
s = 0.03 # in case needs to be set manually, otherwise comment
Modulation(s)

()

# Example codes for experimental measurements

Measuring bitstream (Figure 2c)

In [3]:
Nbits=10000 #depends on how many bits you want to measure at once
N_measurement #number of measurement you want to repeat
for i in range(N_measurement):
    temp=Take_Nbits(Nbits,draw_flag=False, sampling_rate=250000);

Probability measurement with varying waveplate angle (Figure 2b)

In [41]:
total_prob, total_angle=[],[]
N = 12 #depens on how many angle points you want to measure
Nbits=20000 #depends on how many bits you want to measure at once
θ_initial = 0 # initial waveplate angle in degree
r.moveto(θ_initial) # initialize the waveplate angle
θ_step = 15 # unit step of waveplate angle rotation
for i in range(N):
    temp=Take_Nbits(Nbits,draw_flag=False, sampling_rate=250000);
    total_prob.append(np.mean(temp))
    total_angle.append(θ_initial+θ_step*i)
    r.moveby(θ_step)
    time.sleep(3) ## to ensure stable measurement

##result##
plt.figure()
plt.plot(total_angle,total_prob)


Probability measurement with varying time delay between pump and bias (Figure 4)

In [5]:
pm = pylablib.devices.Thorlabs.kinesis.KinesisPiezoMotor("your_device_number") # for case when piezo motor is not connected. Once connected, comment this line
pm_start_position=0 #depends on your current position of piezo motor. This value doesn't tell the absolute position of the piezo stage.
pm.move_to(pm_start_position,channel=1) # piezo stage is controlled via channel of piezo controller.
N_points = 10 #number of points you want to measure the data
unit_step = 3 # number of ticks moving per each movement 
# return position of the piezo stage and probability at that point, and plot the probs-pos figure
(pos, probs) = Piezo_Sweep(N_points, unit_step, pm_start_position=pm_start_position)  
# to reconstruct the pulse shape, you need to know the actual moving distance of the piezo stage per 1 tick.
# time dependent bias field can be obtained by taking inversion erf function on 2*probs-1 (Please refer to figure 4 in the main text)